In [2]:
import numpy as np
import pandas as pd

In [2]:
admissions = pd.read_csv('binary.csv')
admissions

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4
...,...,...,...,...
395,0,620,4.00,2
396,0,560,3.04,3
397,0,460,2.63,2
398,0,700,3.65,2


In [3]:
# make dummy variables for rank
r = admissions['rank']
nr = pd.get_dummies(r, prefix='rank')
nr

,rank_1,rank_2,rank_3,rank_4
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1
...,...,...,...,...
395,0,1,0,0
396,0,0,1,0
397,0,1,0,0
398,0,1,0,0


In [4]:
data = pd.concat([admissions, nr], axis=1)
data = data.drop('rank', axis=1)
data

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,380,3.61,0,0,1,0
1,1,660,3.67,0,0,1,0
2,1,800,4.00,1,0,0,0
3,1,640,3.19,0,0,0,1
4,0,520,2.93,0,0,0,1
...,...,...,...,...,...,...,...
395,0,620,4.00,0,1,0,0
396,0,560,3.04,0,0,1,0
397,0,460,2.63,0,1,0,0
398,0,700,3.65,0,1,0,0


In [5]:
#standardize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:, field] = (data[field]-mean)/std
data

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,-1.798011,0.578348,0,0,1,0
1,1,0.625884,0.736008,0,0,1,0
2,1,1.837832,1.603135,1,0,0,0
3,1,0.452749,-0.525269,0,0,0,1
4,0,-0.586063,-1.208461,0,0,0,1
...,...,...,...,...,...,...,...
395,0,0.279614,1.603135,0,1,0,0
396,0,-0.239793,-0.919418,0,0,1,0
397,0,-1.105469,-1.996759,0,1,0,0
398,0,0.972155,0.683454,0,1,0,0


In [6]:
# Split off random 10% of the data for testing
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.iloc[sample], data.drop(sample)
print(len(data), len(test_data))

360 40


In [7]:
# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']
print(features_test, targets_test)

          gre       gpa  rank_1  rank_2  rank_3  rank_4
20  -0.759199 -0.577822       0       0       1       0
21   0.625884  0.630901       0       1       0       0
48  -1.278605 -2.390908       0       0       0       1
50   0.452749  1.235263       0       0       1       0
54   0.625884 -0.131120       0       0       1       0
58  -1.624876  0.683454       0       1       0       0
71  -2.490553 -1.234737       0       0       0       1
87   0.106478  0.236752       0       1       0       0
99  -1.624876 -0.209950       0       0       1       0
102 -1.798011 -0.157397       0       0       0       1
106  0.972155  0.446965       1       0       0       0
121 -0.932334 -1.891652       0       1       0       0
130  0.279614 -0.577822       0       1       0       0
149  1.318426 -0.209950       1       0       0       0
151 -1.624876 -0.026014       0       1       0       0
160  0.106478 -0.682929       0       1       0       0
169  0.106478  0.604625       0       0       1 

In [10]:
# print features data and it's prediction
print(features.shape, targets.shape)
# print test data and it's prediction
print(features_test.shape, targets_test.shape)

(360, 6) (360,)
(40, 6) (40,)


In [13]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

print(sigmoid(600), sigmoid(0))

1.0 0.5


In [19]:
n_records, n_features = features.shape
last_loss = None

# initialize weights
np.random.seed(42)
weights = np.random.normal(scale=1/n_features**.5, size=n_features)
print(weights)

# neural network hyperparameters
epochs = 1000
learnrate = 0.5

[ 0.2027827  -0.05644616  0.26441774  0.62177434 -0.09559271 -0.09558601]


In [21]:
np.zeros(weights.shape)

array([0., 0., 0., 0., 0., 0.])

In [26]:
for x,y in zip(features_test.values, targets_test):
    print("x: {}, prediction: {}".format(x, y))
    print(sigmoid(np.dot(x, weights)))

x: [-0.75919866 -0.57782239  0.          0.          1.          0.        ], prediction: 0
0.4459793734259117
x: [0.62588442 0.63090111 0.         1.         0.         0.        ], prediction: 1
0.6710815730709019
x: [-1.27860482 -2.39090764  0.          0.          0.          1.        ], prediction: 0
0.445243639930989
x: [0.45274903 1.23526286 0.         0.         1.         0.        ], prediction: 0
0.48163104747784724
x: [ 0.62588442 -0.13112022  0.          0.          1.          0.        ], prediction: 0
0.5096805534034761
x: [-1.62487559  0.68345431  0.          1.          0.          0.        ], prediction: 0
0.5630868132003715
x: [-2.49055251 -1.23473733  0.          0.          0.          1.        ], prediction: 0
0.37029982025007924
x: [0.10647826 0.23675215 0.         1.         0.         0.        ], prediction: 0
0.6524900368620513
x: [-1.62487559 -0.20995002  0.          0.          1.          0.        ], prediction: 0
0.3981358427569469
x: [-1.79801097 -0

In [27]:
for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x, y in zip(features.values, targets):
        # Loop through all records, x is the input, y is the target

        # Activation of the output unit
        #   Notice we multiply the inputs and the weights here 
        #   rather than storing h as a separate variable 
        output = sigmoid(np.dot(x, weights))

        # The error, the target minus the network output
        error = y - output

        # The error term
        #   Notice we calulate f'(h) here instead of defining a separate
        #   sigmoid_prime function. This just makes it faster because we
        #   can re-use the result of the sigmoid function stored in
        #   the output variable
        error_term = error * output * (1 - output)

        # The gradient descent step, the error times the gradient times the inputs
        del_w += error_term * x

    # Update the weights here. The learning rate times the  
    # change in weights, divided by the number of records to average
    weights += learnrate * del_w / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss


# Calculate accuracy on test data
tes_out = sigmoid(np.dot(features_test, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Train loss:  0.26276093849966364
Train loss:  0.20928619409324895
Train loss:  0.20084292908073417
Train loss:  0.1986215647552789
Train loss:  0.19779851396686018
Train loss:  0.19742577912189863
Train loss:  0.19723507746241067
Train loss:  0.19712945625092465
Train loss:  0.19706766341315074
Train loss:  0.19703005801777368
Prediction accuracy: 0.725


In [7]:
np.mean([1, 1] == [1, 1])

1.0

In [10]:
np.mean(0.4 == 0.4)

1.0